In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nltk.classify.util

import nltk
import string

from collections import Counter

from astropy.table import Table, Column
from datetime import timedelta

In [2]:
data = pd.read_csv('Amazon.csv')
print(data.shape)
data.head(5)

(455000, 13)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpScore,helpful
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,I was looking forward to try cranberry apple f...,0.5,False
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,"TY for everything. The seeds arrived quickly,...",NaN,False
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,I've finally found the best cereal in the worl...,NaN,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,I originally bought these chips because I'd he...,1.0,False
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,"Really excellent tea, flowers are visible in t...",NaN,False


In [3]:
#fix Summary
data.Summary = data.Summary.fillna('na')
##GETTING A REPORT OF MISSING VALUES
data.isnull().sum()
data.head(5)

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpScore,helpful
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,I was looking forward to try cranberry apple f...,0.5,False
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,"TY for everything. The seeds arrived quickly,...",NaN,False
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,I've finally found the best cereal in the worl...,NaN,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,I originally bought these chips because I'd he...,1.0,False
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,"Really excellent tea, flowers are visible in t...",NaN,False


In [4]:
# features from Amazon.csv to add to feature set

#features related to text body of review
data['reviewLen'] = data['Text'].str.len()
data['hasEP'] = data['Text'].str.contains('!')
data['hasQM'] = data['Text'].str.contains('\?')
data['hasAST'] = data['Text'].str.contains('\*')
data['hasDS'] = data['Text'].str.contains('$')
data['hasSC'] = data['Text'].str.contains(';')
data['hasAT'] = data['Text'].str.contains('@')
data['bodyNum'] = data['Text'].str.contains('\d')

data['EPcount'] = data['Text'].str.count('!')
data['QMcount'] = data['Text'].str.count('\?')
data['ASTcount'] = data['Text'].str.count('\*')
data['DScount'] = data['Text'].str.count('$')
data['SCcount'] = data['Text'].str.count(';')
data['ATcount'] = data['Text'].str.count('@')
data['NUMcount'] = data['Text'].str.count('\d')



#features related to profile name 
data['nameLen'] = data['ProfileName'].str.len()
data['nameNum'] = data['ProfileName'].str.contains('\d')
data['nameNumCount'] = data['ProfileName'].str.count('\d')
data['nameQUOTE'] = data['ProfileName'].str.contains('"')
data['nameSPACE'] = data['ProfileName'].str.contains(' ')


#time? sooner the better? 
data['first'] = data['Time'].rank(ascending=1)

#not sure if this is the best way to go about doing this - 
#finding repeating names 
data['repeatUser'] = data.duplicated('ProfileName')
data['repeatSum'] = data.duplicated('Summary')
data['repeatText'] = data.duplicated('Text')

#add Summary related features
data['sumLen'] = data['Summary'].str.len()
data['sumNum'] = data['Summary'].str.contains('\d')
data['sumNumCount'] = data['Summary'].str.count('\d')
data['sumQUOTE'] = data['Summary'].str.contains('"')
data['sumQUEST'] = data['Summary'].str.contains('\?')
data['sumEP'] = data['Summary'].str.count('!')

#is entire text in capitals? 
data['sumCapBool'] = data['Summary'].str.isupper()
data['nameCapBool'] = data['ProfileName'].str.isupper()
data['capBool'] = data['Text'].str.isupper()

data.tail(10)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,...,repeatText,sumLen,sumNum,sumNumCount,sumQUOTE,sumQUEST,sumEP,sumCapBool,nameCapBool,capBool
454990,320405,320406,B000E1HVR0,A3CO3BDS4PXKL5,E. Morris,0,0,5,1249171200,OMG... need I say more,...,False,22,False,0,False,False,0,False,False,False
454991,104610,104611,B000KSTY86,A3DTJW2ZSDQZ86,M. B. Goetz,11,11,5,1270080000,Great coffee - Great price,...,False,26,False,0,False,False,0,False,False,False
454992,282272,282273,B000FICDO8,A3B5OLV01PVP85,"D. Patel ""SpeedReader""",1,1,5,1258675200,Great taste with little effort.,...,False,31,False,0,False,False,0,False,False,False
454993,419683,419684,B0029ZAOW8,A2HAVA4SPL9HSM,"Larry A. Chrispyn ""PUBLISHED POET""",0,2,1,1329696000,PRICE TOO HIGH NO EFFECT,...,True,28,False,0,False,False,0,True,False,True
454994,115270,115271,B003E50WMY,A37FP7HWWNTW57,Gary,0,0,5,1314921600,My Poinion.,...,False,11,False,0,False,False,0,False,False,False
454995,457613,457614,B008Z5L2MW,A2W7HYIQWAJ091,"Bonnie Pierce ""TexasBonnieBelle""",0,0,5,1349049600,Outstanding - what else?,...,False,24,False,0,False,True,0,False,False,False
454996,283509,283510,B0045Z4JAI,AZ3GK5ZX3SVKT,gsue,2,2,2,1306195200,"Too weak, not enough umph!",...,True,26,False,0,False,False,1,False,False,False
454997,292569,292570,B001BM68S4,A359CZWDE8MPMF,LC,0,0,5,1314576000,Good Stuff,...,False,10,False,0,False,False,0,False,True,False
454998,156159,156160,B002YLG5MA,A1V8J8FKBZW68K,"Ansella Adams ""Valley Lady""",4,4,5,1275350400,Way to Start the Day!,...,False,21,False,0,False,False,1,False,False,False
454999,119066,119067,B004IREFUM,A1MYYCC9BSYV5Y,Colorado Mom,0,0,5,1338422400,Delicious,...,True,9,False,0,False,False,0,False,False,False


In [5]:
# Count number of capital letters in Summary, Text, and Name
def capital(word):
    num = 0
    for i in word:
            if i.isupper():
                 num+=1
    return num

data['sumCapCount'] = data['Summary'].apply(capital)
data['textCapCount'] = data['Text'].apply(capital)
data['textC2L'] = (data['textCapCount'] / data['reviewLen'])
data['sumC2L'] = (data['sumCapCount'] / data['sumLen'])


data.tail(3)

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,...,sumQUOTE,sumQUEST,sumEP,sumCapBool,nameCapBool,capBool,sumCapCount,textCapCount,textC2L,sumC2L
454997,292569,292570,B001BM68S4,A359CZWDE8MPMF,LC,0,0,5,1314576000,Good Stuff,...,False,False,0,False,True,False,2,16,0.056537,0.200000
454998,156159,156160,B002YLG5MA,A1V8J8FKBZW68K,"Ansella Adams ""Valley Lady""",4,4,5,1275350400,Way to Start the Day!,...,False,False,1,False,False,False,3,8,0.030075,0.142857
454999,119066,119067,B004IREFUM,A1MYYCC9BSYV5Y,Colorado Mom,0,0,5,1338422400,Delicious,...,False,False,0,False,False,False,1,12,0.043636,0.111111


In [6]:
#turns colum into vector - here columns 7 and 13
XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)
XreviewLen = data.iloc[:, 13].values.reshape(data.shape[0], 1)
XhasEP = data.iloc[:, 14].values.reshape(data.shape[0], 1)
XhasQM = data.iloc[:, 15].values.reshape(data.shape[0], 1)
XhasAST = data.iloc[:, 16].values.reshape(data.shape[0], 1)
XhasDS = data.iloc[:, 17].values.reshape(data.shape[0], 1)
XhasSC = data.iloc[:, 18].values.reshape(data.shape[0], 1)
XhasAT = data.iloc[:, 19].values.reshape(data.shape[0], 1)
XBodyNum = data.iloc[:, 20].values.reshape(data.shape[0], 1)
XEPcount = data.iloc[:, 21].values.reshape(data.shape[0], 1)
XQMcount = data.iloc[:, 22].values.reshape(data.shape[0], 1)
XASTcount = data.iloc[:, 23].values.reshape(data.shape[0], 1)
XDScount = data.iloc[:, 24].values.reshape(data.shape[0], 1)
XSCcount = data.iloc[:, 25].values.reshape(data.shape[0], 1)
XATcount = data.iloc[:, 26].values.reshape(data.shape[0], 1)
XNUMcount = data.iloc[:, 27].values.reshape(data.shape[0], 1)

XnameLen = data.iloc[:, 28].values.reshape(data.shape[0], 1)
XnameNum = data.iloc[:, 29].values.reshape(data.shape[0], 1)
XnameNumCount = data.iloc[:, 30].values.reshape(data.shape[0], 1)
XnameQUOTE = data.iloc[:, 31].values.reshape(data.shape[0], 1)
XnameSPACE = data.iloc[:, 32].values.reshape(data.shape[0], 1)
Xfirst = data.iloc[:, 33].values.reshape(data.shape[0], 1)

XrepeatUser = data.iloc[:, 31].values.reshape(data.shape[0], 1)
XrepeatSum = data.iloc[:, 32].values.reshape(data.shape[0], 1)
XrepeatText = data.iloc[:, 33].values.reshape(data.shape[0], 1)

XsumLen = data.iloc[:, 34].values.reshape(data.shape[0], 1)
XsumNum = data.iloc[:, 35].values.reshape(data.shape[0], 1)
XsumNumCount = data.iloc[:, 36].values.reshape(data.shape[0], 1)
XsumQUOTE = data.iloc[:, 37].values.reshape(data.shape[0], 1)
XsumQUEST = data.iloc[:, 38].values.reshape(data.shape[0], 1)
XsumEP = data.iloc[:, 39].values.reshape(data.shape[0], 1)

XsumCapBool = data.iloc[:, 40].values.reshape(data.shape[0], 1)
XnameCapBool = data.iloc[:, 41].values.reshape(data.shape[0], 1)
XcapBool = data.iloc[:, 42].values.reshape(data.shape[0], 1)

XsumCapCount = data.iloc[:, 43].values.reshape(data.shape[0], 1)
XtextCapCount = data.iloc[:, 44].values.reshape(data.shape[0], 1)
XtextC2L = data.iloc[:, 45].values.reshape(data.shape[0], 1)
XsumC2L = data.iloc[:, 46].values.reshape(data.shape[0], 1)

    

Xtoadd = np.concatenate((XScore, XreviewLen, XhasEP, XhasQM, XhasAST, XhasDS, XhasSC, XhasAT, XBodyNum, XEPcount, XQMcount, XASTcount, XDScount, XSCcount, XATcount, XNUMcount, 
XnameLen, XnameNum, XnameNumCount, XnameQUOTE, XnameSPACE, Xfirst,XrepeatUser, XrepeatSum, XrepeatText, XsumLen,XsumNum, XsumNumCount, XsumQUOTE, XsumQUEST, XsumEP, XsumCapBool, XnameCapBool, XcapBool, XsumCapCount, XtextCapCount,XtextC2L, XsumC2L ), axis=1)

In [7]:
# report on training and test sets
global error, acc, tp, tn

#class SVM:
#    def __init__(self, error):
#        self.error = erTEST

def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    TruNegTRAIN = (((y_train==False) & (y_pred==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', TruNegTRAIN, TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    
      
    #t.show_in_browser(jsviewer=True) 
    



In [ ]:
#def all_models_table():
#    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
#            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
#            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
#            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
#    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
#    print(tt)
    

In [8]:
# vectorize Bag of Words from review text; as sparse matrix
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(min_df=1, ngram_range=(1,3))
X = tv.fit_transform(data.Text)

#if you restrict it to 2 to the power of 17 features is a few extra, way to restrict number of features created 

In [9]:
# vectorize Bag of Words from Summary text; 
# from sklearn.feature_extraction.text import TfidfVectorizer
tvsum = TfidfVectorizer(min_df=1, ngram_range=(1,2))
Xsum = tvsum.fit_transform(data.Summary)

In [10]:
# convert additional features to sparse matrix and concatenate onto the bag of words sparse matrix
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)
Xfinal = hstack([X, Xsum, XtoaddSparse])
X = csr_matrix(Xfinal)

TypeError: Cannot cast array data from dtype('O') to dtype('bool') according to the rule 'safe'

In [11]:
# size of feature set
print(X.shape)

(455000, 12918020)


In [12]:
# define y
y = data.iloc[:, 12].values
y.shape

(455000,)

In [13]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [14]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [15]:
# MODEL: SVM, linear
#will also go through each row as many times as you want it to - default is 5, making changes to weights, trying to
#move them closer and closer to lessen cost i think

from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

#getting only 25% of positives correctly classified 

     SVM          Training Set      Test Set   
-------------- ----------------- --------------
    Error Rate 0.000841444270016 0.150366300366
 Accuracy Rate     0.99915855573 0.849633699634
True Positives    0.993585328052 0.453882282402
True Negatives    0.999596981766 0.880942028413


In [16]:
# MODEL: logistic regression
from sklearn import linear_model
#alpha, heavily penalizes the weights 
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set      Test Set   
------------------- ----------------- --------------
         Error Rate 0.000753532182104 0.147142857143
      Accuracy Rate    0.999246467818 0.852857142857
     True Positives    0.994618563802 0.450184870591
     True Negatives    0.999610528597  0.88471298807


In [17]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set   
-------------- ---------------- ---------------
    Error Rate 0.00187441130298 0.0760366300366
 Accuracy Rate   0.998125588697  0.923963369963
True Positives   0.999311176167  0.324073148796
True Negatives    0.99803232274  0.971421343474


In [18]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set      Test Set   
-------------- ----------------- --------------
    Error Rate 0.000919937205651 0.151868131868
 Accuracy Rate    0.999080062794 0.848131868132
True Positives    0.994360254865 0.453282702109
True Negatives    0.999451353328 0.879368818828


In [19]:
# # Ridge
# from sklearn.linear_model import Ridge
# regr = linear_model.Ridge()
# regr.fit(X_train_std, y_train)
# y_pred = regr.fit(X_train_std, y_train).predict(X_train_std)
# y_pred_test = regr.predict(X_test_std)
# print_results('Perceptron')